# Selenium이용한 크롤링실습

In [53]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [54]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [55]:
service = Service(executable_path=ChromeDriverManager().install())

## 싼 주유소 찾기 Opinet

- https://www.opinet.co.kr/user/main/mainView.do

### 서울시 구별 주유소 가격 정도 얻어오기

- 사이트에서 [싼주유소찾기] > [지역별] 메뉴 선택
- 지역은 '서울' > 원하는 구 지정
    - '시/군/구'를 선택하는 select option의 XPath 찾기

- '조회'버튼 클릭
- 조회된 주유소 정보 '엑셀저장' 버튼 클릭하여 엑셀파일 다운로드

#### 크롬 드라이버 객체 생성

In [56]:
driver = webdriver.Chrome(service=service)

#### 웹페이지 로드

In [5]:
driver.implicitly_wait(3)

In [7]:
url = 'https://www.opinet.co.kr/user/main/mainView.do'
driver.get(url)

#### [싼주유소찾기]-[지역] 페이지로 이동

- 자바스트립트 코드 실행으로 통해서 이동

In [8]:
driver.execute_script('goSubPage(0,0,99)')

#### 구 이름 찾기

In [10]:
#시군구 이름 찾기 - 방법1
xpath = '//*[@id="SIGUNGU_NM0"]'
gu_lists = driver.find_element(By.XPATH, xpath)
gu_opt = gu_lists.find_elements(By.TAG_NAME, 'option')

gu_names = []
for opt in gu_opt:
    gu_names.append(opt.get_attribute('value'))
# gu_names

In [25]:
#시군구 이름 찾기 - 방법2
xpath = '//*[@id="SIGUNGU_NM0"]'
gu_lists = driver.find_element(By.XPATH, xpath)
gu_opt = gu_lists.find_elements(By.TAG_NAME, 'option')
gu_names = [opt.get_attribute('value') for opt in gu_opt][1:]
# gu_names

In [24]:
#시도 이름 찾기
xpath = '//*[@id="SIDO_NM0"]'
sido_lists = driver.find_element(By.XPATH, xpath)
sido_opt = sido_lists.find_elements(By.TAG_NAME, 'option')
sido_names = [opt.get_attribute('value') for opt in sido_opt][1:]
#sido_names

#### 서울시 찾기 / '강남구' 주유소 지정

In [18]:
#시도 이름 선택
element = driver.find_element(By.ID, 'SIDO_NM0')
element.send_keys('서울특별시')

In [19]:
#시군구 이름 선택
element = driver.find_element(By.ID, 'SIGUNGU_NM0')
element.send_keys('강남구')

#### '조회' 버튼 클릭

In [20]:
xpath = '//*[@id="searRgSelect"]'
driver.find_element(By.XPATH, xpath).click()

#### 조회된 주유소 '엑셀저장' 버튼 클릭

In [22]:
xpath = '//*[@id="glopopd_excel"]'
driver.find_element(By.XPATH, xpath).click()

=> '지역_위치별(주유소).xls'파일 다운로드 폴더에 저장됨

### 25개 구 주유소 엑셀파일 다운로드

- 함수로 작성하여 실행

In [29]:
import time

#서울시 휘발유 가격 엑셀파일 다운로드 함수
# 시군구 주유소 검색 후 엑셀 다운로드
def search_download_sigungu(sigungu):
    #시군구 이름 선택
    element = driver.find_element(By.ID, 'SIGUNGU_NM0')
    element.send_keys(sigungu)
    time.sleep(2)

    #조회 버튼 클릭
    xpath = '//*[@id="searRgSelect"]'
    driver.find_element(By.XPATH, xpath).click()
    time.sleep(2)

    #엑셀저장 버튼 클릭
    xpath = '//*[@id="glopopd_excel"]'
    driver.find_element(By.XPATH, xpath).click()
    time.sleep(2)


#입력한 시도의 모든 구군별 주유소 검색 후 엑셀 다운로드
def search_download_sido(sido):

    #시도 이름 선택
    element = driver.find_element(By.ID, 'SIDO_NM0')
    element.send_keys(sido)
    time.sleep(2)
    
    #시군구 이름 찾기
    xpath = '//*[@id="SIGUNGU_NM0"]'
    gu_lists = driver.find_element(By.XPATH, xpath)
    gu_opt = gu_lists.find_elements(By.TAG_NAME, 'option')
    gu_names = [opt.get_attribute('value') for opt in gu_opt][1:]
        
    #구군 조회 후 엑셀 다운로드
    for sigungu in gu_names:
        #구군 조회 후 엑셀 다운로드
        search_download_sigungu(sigungu)


#전체 시도별 주유소 가격 다운로드
def search_all():
    #시도명 찾기
    xpath = '//*[@id="SIDO_NM0"]'
    sido_lists = driver.find_element(By.XPATH, xpath)
    sido_opt = gu_lists.find_elements(By.TAG_NAME, 'option')
    sido_names = [opt.get_attribute('value') for opt in sido_opt][1:]

    #시도별 주유소 가격 다운로드
    for sido in sido_names:
        search_download_sido(sido)

In [30]:
search_download_sido('서울특별시')

### 엑셀 파일 읽어오기

In [43]:
from glob import glob
#glob 함수 : 파일 경로 및 이름을 모아서 리스트에 저장
# * 문자 사용

In [37]:
import os
cpath = os.getcwd()
cpath

'/Users/crystal.moon/crystal.moon/Python/04_crawling'

In [45]:
path = '/Users/crystal.moon/Downloads/'
os.listdir(path)

['지역_위치별(주유소) (9).xls',
 '지역_위치별(주유소) (12).xls',
 '지역_위치별(주유소) (5).xls',
 '.DS_Store',
 '지역_위치별(주유소) (4).xls',
 '지역_위치별(주유소) (13).xls',
 '지역_위치별(주유소) (8).xls',
 '.localized',
 '지역_위치별(주유소) (3).xls',
 '지역_위치별(주유소) (18).xls',
 '지역_위치별(주유소) (22).xls',
 '지역_위치별(주유소) (14).xls',
 '지역_위치별(주유소) (15).xls',
 '지역_위치별(주유소) (23).xls',
 '지역_위치별(주유소) (19).xls',
 '지역_위치별(주유소) (2).xls',
 '지역_위치별(주유소) (20).xls',
 '지역_위치별(주유소) (1).xls',
 '지역_위치별(주유소) (16).xls',
 '지역_위치별(주유소) (17).xls',
 '지역_위치별(주유소) (21).xls',
 '지역_위치별(주유소) (10).xls',
 '지역_위치별(주유소).xls',
 '지역_위치별(주유소) (7).xls',
 '지역_위치별(주유소) (6).xls',
 '지역_위치별(주유소) (11).xls']

In [46]:
# * : all 이라는 의미 
filename = '지역_위치별*.xls'
# path + filename
files = glob(path+filename)
len(files)

24

In [50]:
files[0]

'/Users/crystal.moon/Downloads/지역_위치별(주유소) (9).xls'

In [58]:
# xlrd upgrade 요청 에러
!pip install xlrd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.6 MB/s eta 0:00:00


In [61]:
station = pd.read_excel(files[0], header=2)
type(station)

pandas.core.frame.DataFrame

### 데이터 합치기

In [63]:
tmp = []
for file in files:
    station = pd.read_excel(file, header=2)
    tmp.append(station)
stations = pd.concat(tmp)
stations

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,(주)자연에너지 국민주유소,서울 동대문구 답십리로 223 (답십리동),HD현대오일뱅크,02-2216-7155,Y,1820,1597,1497,-
1,서울특별시,동서울고속주유소,서울 동대문구 한천로 414,SK에너지,02-969-1144,Y,1820,1597,1497,-
2,서울특별시,재정제2주유소,서울 동대문구 사가정로 90 (전농동),HD현대오일뱅크,02-2249-6682,Y,-,1607,1507,-
3,서울특별시,재정주유소,서울 동대문구 전농로 121 (전농동),HD현대오일뱅크,070-8256-4617,Y,1830,1607,1507,-
4,서울특별시,풍한주유소,서울 동대문구 안암로 168,SK에너지,02-924-5189,N,-,1615,1515,-
...,...,...,...,...,...,...,...,...,...,...
7,서울특별시,에스에스오토셀프,서울 마포구 서강로 76 (창전동),HD현대오일뱅크,02-336-5185,Y,2099,1689,1619,-
8,서울특별시,(주)안국에너지 망원동주유소,서울 마포구 월드컵로 119 (망원동),SK에너지,02-334-0917,N,-,1698,1598,-
9,서울특별시,(주)만성상사 대흥주유소,서울 마포구 대흥로 61 (대흥동),SK에너지,02-3273-5151,N,2149,1699,1629,1745
10,서울특별시,SK에너지 양지주유소,서울 마포구 마포대로 69 (도화동),SK에너지,02-706-8955,N,1919,1699,1659,-


In [64]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, 0 to 11
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      420 non-null    object
 1   상호      420 non-null    object
 2   주소      420 non-null    object
 3   상표      420 non-null    object
 4   전화번호    420 non-null    object
 5   셀프여부    420 non-null    object
 6   고급휘발유   420 non-null    object
 7   휘발유     420 non-null    object
 8   경유      420 non-null    object
 9   실내등유    420 non-null    object
dtypes: object(10)
memory usage: 36.1+ KB


In [65]:
#csv 저장
stations.to_csv('주유소.csv')

In [67]:
#driver.close()

-----